In [19]:
# !pip install wwo-hist

  Created wheel for wwo-hist: filename=wwo_hist-0.0.4-py3-none-any.whl size=4218 sha256=515dbb8c6fb887f31d6419bbcdabf80f12d36dd7c177d77efe7a4ea63e65117f
  Stored in directory: c:\users\remek\appdata\local\pip\cache\wheels\47\67\a6\977ac091198b98e5133420721fc4d04575c8db9564e02c8759
Successfully built wwo-hist


In [20]:
# uzyskanie 'prognozy' danych pogodowych wg https://github.com/MichalPorwisz/warsaw-pollution

import os

from wwo_hist import retrieve_hist_data

frequency = 1
start_date = '25-DEC-2014'
end_date = '10-JAN-2020'
# TODO: Put your key
api_key = '' 
location_list = ['warsaw']
# TODO: inside they filter out some columns - maybe try seeing all
hist_weather_data = retrieve_hist_data(api_key,
                                location_list,
                                start_date,
                                end_date,
                                frequency,
                                location_label = False,
                                export_csv = True,
                                store_df = True)
print('done')

# path like this because chdir changed directory
hist_weather_data[0].to_csv('warsaw_full.csv')



Retrieving weather data for warsaw


Currently retrieving data for warsaw: from 2014-12-25 to 2014-12-31
Time elapsed (hh:mm:ss.ms) 0:00:00.851697
Currently retrieving data for warsaw: from 2015-01-01 to 2015-01-31
Time elapsed (hh:mm:ss.ms) 0:00:02.514303
Currently retrieving data for warsaw: from 2015-02-01 to 2015-02-28
Time elapsed (hh:mm:ss.ms) 0:00:03.819765
Currently retrieving data for warsaw: from 2015-03-01 to 2015-03-31
Time elapsed (hh:mm:ss.ms) 0:00:04.935783
Currently retrieving data for warsaw: from 2015-04-01 to 2015-04-30
Time elapsed (hh:mm:ss.ms) 0:00:06.627262
Currently retrieving data for warsaw: from 2015-05-01 to 2015-05-31
Time elapsed (hh:mm:ss.ms) 0:00:07.890920
Currently retrieving data for warsaw: from 2015-06-01 to 2015-06-30
Time elapsed (hh:mm:ss.ms) 0:00:09.214381
Currently retrieving data for warsaw: from 2015-07-01 to 2015-07-31
Time elapsed (hh:mm:ss.ms) 0:00:10.539838
Currently retrieving data for warsaw: from 2015-08-01 to 2015-08-31
Time elapsed 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

import gc
import pickle
from os import path

np.random.seed(0)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer, accuracy_score
from ml_metrics import rmse

from functools import partial
from hyperopt import hp
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
def beginFE():
    
    for one in globals().copy().keys():
        if one.endswith('_feats'):
            del globals()[one]
    if 'feats' in globals():
        del globals()['feats']
    
    train = pd.read_hdf('train_warsaw.h5')
    train = train.rename(columns={'timestamp': 'date'}).reset_index()
    train.drop( columns=['timestamp'], inplace = True)
    test = pd.read_hdf('test_warsaw.h5')
    test = test.rename(columns={'timestamp': 'date'}).reset_index()
    test.drop( columns=['timestamp'], inplace = True)
    
    df = pd.concat([train, test], sort=True)
    df.sort_values(['date', 'id'], inplace=True)
    
    # fragment kodu dodający prognozę pogody - proste użycie tych cech i ich agregatów pogorszyło wynik
    
#     weather_forrecast = pd.read_csv('warsaw_full.csv')
#     weather_forrecast['date_time'] = pd.to_datetime(weather_forrecast['date_time'], format="%Y-%m-%d %H:%M:%S")
    
#     df = pd.merge(df, weather_forrecast, left_on='date', right_on='date_time')
#     df.drop( columns=['Unnamed: 0', 'date_time', 'moonrise', 'moonset', 'sunrise', 'sunset'], inplace = True)
    
#     forrecast_feats = ['maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex_y', 'uvIndex.1', 'moon_illumination', 
#                        'DewPointC', 'FeelsLikeC', 'HeatIndexC', 'WindChillC', 'WindGustKmph', 'cloudcover', 'humidity_y', 
#                        'precipMM', 'pressure_y', 'tempC', 'visibility_y', 'winddirDegree', 'windspeedKmph']

    forrecast_feats = []
        
    df['hour']    = df.date.dt.hour
    df['day_m']   = df.date.dt.day
    df['month']   = df.date.dt.month
    df['year']    = df.date.dt.year
    df['week_y']  = df.date.dt.week
    df['day_y']   = df.date.dt.dayofyear
    df['day_w']   = df.date.dt.dayofweek
    df['quarter'] = df.date.dt.quarter
    df['hour_w']  = df.date.dt.hour + df.date.dt.dayofweek * 24
    
    df['pm25_log']      = (np.log(df[ df['pm25']>0 ]['pm25']+1))
    
    time_feats = ['quarter', 'month', 'week_y', 'day_y', 'day_m', 'day_w', 'hour', 'hour_w', 'year']

    black_list_feats = ['id', 'pm25', 'pm25_log', 'is_non_cont', 'id_dif', 'd_time', 'cont_nr']

    return df, time_feats, black_list_feats, forrecast_feats
    
def endFE(df, black_list_feats):

    # listy kawałków
    def count_cont_nr(flag):
        global value
        if flag:
            value = 1
        else:
            value += 1
        return int(value)
    
    df['is_non_cont'] = df.shift(1).id != (df.id - 1)
    df['id_dif'] = (df.id - df.shift(1).id).fillna(0).astype(int)
    df['d_time'] = df.date - df.shift(1).date
    df.reset_index(drop=True, inplace=True)
    df['cont_nr'] = df.apply(lambda x: count_cont_nr(x['is_non_cont']), axis=1)
    begin_idx = df.loc[df['is_non_cont']].index.values.astype(int)
    end_idx = (df.shift(1).loc[df['is_non_cont']]['cont_nr'].shift(-1) + 
               df.loc[df['is_non_cont']].index.values).values.astype(int)
    end_idx[-1] = int(len(df))
    parts = []
    for i in range(len(begin_idx)):
        parts.append( df.iloc[ begin_idx[i] : (end_idx[i]) ].copy() )
        
    train, test = df[ ~df.pm25.isnull() ], df[ df.pm25.isnull() ]
    
    num_feats = df.select_dtypes(include=[np.number, bool]).columns.values
    num_feats = [feat for feat in num_feats if feat not in black_list_feats]
    omit_feats = [x for x in df.columns if x not in num_feats]
            
#     df.fillna(-1, inplace=True)
#     df = df.iloc[24:,]
    
    return df, train, test, parts, num_feats, omit_feats

In [3]:
def feature_engineering_aggr(avg_len=6, roll_nr=[6, 12], agr_fn=['min', 'max', 'mean', 'median']):
    
    df, time_feats, black_list_feats, forrecast_feats = beginFE()
    
    df['windSpeed_log'] = (np.log(df[ df['windSpeed']>0 ]['windSpeed']+5))
    df['windGust_log']  = (np.log(df[ df['windGust']>0  ]['windGust']+2))
    
    obj_feats = df.select_dtypes(np.object).columns
    for feat in obj_feats:
        df[feat + '_cat'] = df[feat].factorize()[0]
        
    past_feats = df.select_dtypes(include=[np.number, bool]).columns.values
    past_feats = [feat for feat in past_feats if feat not in black_list_feats]
    past_feats = [feat for feat in past_feats if feat not in time_feats]
    
    roll_columns = [feat for feat in past_feats if not feat.endswith('_cat')]   
    
    for r_feat in roll_columns:
        for fn in agr_fn:
            for nr in roll_nr:    
                suffix = '_roll{}{}'.format(nr, fn)
                df[r_feat + suffix] = getattr(df[r_feat].rolling(nr), fn)()
    roll_feats = [feat for feat in df.columns if '_roll' in feat]
        
    df['avg_mov'] = df['pm25'].rolling( window=avg_len, min_periods=1).mean()
    df['avg_mov'] = df.apply(lambda x: np.nan if np.isnan(x['pm25']) else x['avg_mov'], axis=1)
    df['avg_diff'] = df.apply(lambda x: x['pm25']-x['avg_mov'], axis=1)
    
#     df['avg_diff'][df['avg_diff'] > 30] = 30                                   # cap value
#     df['avg_diff'][df['avg_diff'] < -30] = -30
    
    time_feats += forrecast_feats
    
    df, train, test, parts, num_feats, omit_feats = endFE(df, black_list_feats)

    return df, train, test, parts, num_feats, omit_feats, time_feats, black_list_feats

In [4]:
# możliwość podania dowolnej ilosci punktów do predykcji
# bez uwzglednienia przesunięcia cech

def run_model_nr(nr, model_selected, model_params, parts_to_go, feats_selected, y_log):
    
    train_df, y_train_all = pd.DataFrame(columns=feats_selected), np.array([])
    last_days_df, y_last_days, y_last_pred, y_pred_all = train_df.copy(), np.array([]), np.array([]), np.array([])
    models, global_scores, last_day_scores = [], np.array([]), np.array([])

    for i in np.arange(nr):
        models.append(model_selected(**model_params))          # odzielne modele dla 24h
    
    for train in tqdm(parts_to_go):                               # kolejne bloki danych
  
        X = train[feats_selected].values
        y = (train['pm25'].values).astype(np.float64)
        if y_log:
            y = (train['pm25_log'].values).astype(np.float64)
   
        X_last  = X[-25:-24]                                    # ostatni wiersz z bloku danych
        X_train = X[0:-25]                                      # pomija zbiór testowy i ostatni wiersz
        
        for i in np.arange(nr):                                 # każdy model szkolony oddzielnie
            y = pd.Series(y).shift(-1).values                   # przesuwa pm25 o 1h w przeszłość
            y_train = y[0:-25]                       
            models[i].fit(X_train, y_train)                        
            y_last = (models[i].predict(X_last)).astype(np.float64)
            y[-25] = y_last[0]                                  # prognoza na 1h
        
        y_pred = y[-nr-24:-24]                                  # prognoza na 24h
        if y_log:
            y_pred = np.exp(y_pred)-1
         
        #last day CV
        
        X_last_day = X[-nr-25:-nr-24]                           # godzina poprzedzająca ostatni dzień z danymi
        y_last_day = train.pm25[-nr-25:-25].values
        y_last_p = np.array([])
        for i in np.arange(nr):                     
            y_hour = (models[i].predict(X_last_day)).astype(np.float64)
            y_last_p = np.concatenate((y_last_p, y_hour))
        if y_log:
            y_last_p = np.exp(y_last_p)-1
        last_day_scores = np.append( last_day_scores, [rmse(y_last_day, y_last_p)] )
            
        train_df     = train_df + train[feats_selected][0:-25]
        last_days_df = last_days_df + train[feats_selected][-nr-25:-25]
        y_train_all  = np.concatenate((y_train_all, train.pm25[0:-25].values))
        y_last_days  = np.concatenate((y_last_days, y_last_day))
        y_last_pred  = np.concatenate((y_last_pred, y_last_p))
        y_pred_all   = np.concatenate((y_pred_all, y_pred))
        
        # CV
        
        side = 2  # pozwala uniknąć cyklicznych podobieństw parametru - dla 2 sprawdza 5 sąsiednich wartosci
        part_scores = np.array([])
        for i in np.arange(nr):                         # waliduje każdy model osobno
            scores = np.array([])
            for shift in np.arange(-i-side, -i+side):   # przesunięcia wykresu - kiedy bład będzie najmniejszy?
                X_cv = train[feats_selected][0:-25].values
                y_cv = train.pm25[0:-25].values
                if y_log:
                    y_cv = train.pm25_log[0:-25].values
                y_cv = pd.Series(y_cv).shift(shift).values    # przesuwa y w przeszłośc (wartości ujemne)
                y_pred_cv = (models[i].predict(X_cv)).astype(np.float64)
                if y_log: 
                    y_cv = np.exp(y_cv)-1
                    y_pred_cv = np.exp(y_pred_cv)-1
                scores = np.append( scores, [rmse(y_cv[side:-side-nr], y_pred_cv[side:-side-nr])] )
            part_scores = np.append( part_scores, [scores[scores.argmin()]] )
#         print(part_scores.mean(), part_scores.std())

        global_scores = np.append( global_scores, part_scores )
    r2_total = r2_score(y_last_days, y_last_pred)
    rmse_total = rmse(y_last_days, y_last_pred)
    print('shift_score: {:.3f}, last_24_score: {:.3f}, rmse: {:.3f}, r2: {:.3f}\n'.format(global_scores.mean(),
                                                                                        last_day_scores.mean(),
                                                                                        rmse_total, r2_total))      
    return (global_scores, last_day_scores, rmse_total, r2_total, models, y_pred_all, train_df, last_days_df, 
            y_train_all, y_last_days, y_last_pred, feats_selected)

In [5]:
# pomija ostatni okres w zbiorze testowym i używa go do walidacji
# bez uwzglednienia przesunięcia cech

def run_model_day(nr, model_selected, model_params, parts_to_go, feats_selected, y_log):
    
    train_df, y_train_all = pd.DataFrame(columns=feats_selected), np.array([])
    last_days_df, y_last_days, y_last_pred, y_pred_all = train_df.copy(), np.array([]), np.array([]), np.array([])
    models, global_scores, last_day_scores = [], np.array([]), np.array([])

    for i in np.arange(nr):
        models.append(model_selected(**model_params))          # odzielne modele dla 24h
    
    for train in tqdm(parts_to_go):                               # kolejne bloki danych
  
        X = train[feats_selected].values
        y = (train['pm25'].values).astype(np.float64)
        if y_log:
            y = (train['pm25_log'].values).astype(np.float64)
   
        X_last  = X[-25:-24]                                    # ostatni wiersz z bloku danych
        X_train = X[0:-nr-25]                                   # pomija zbiór testowy, ostatni wiersz i ostatni dzień (CV)
        
        for i in np.arange(nr):                                 # każdy model szkolony oddzielnie
            y = pd.Series(y).shift(-1).values                   # przesuwa pm25 o 1h w przeszłość
            y_train = y[0:-nr-25]                       
            models[i].fit(X_train, y_train)                        
            y_last = (models[i].predict(X_last)).astype(np.float64)
            y[-25] = y_last[0]                                  # prognoza na 1h
        
        y_pred = y[-nr-24:-24]                                     # prognoza na 24 h (nr)
        if y_log:
            y_pred = np.exp(y_pred)-1
        
        #last day CV
        
        X_last_day = X[-nr-25:-nr-24]                                 # godzina poprzedzająca ostatni dzień z danymi
        y_last_day = train.pm25[-nr-25:-25].values
        y_last_p = np.array([])
        for i in np.arange(nr):                     
            y_hour = (models[i].predict(X_last_day)).astype(np.float64)
            y_last_p = np.concatenate((y_last_p, y_hour))
        if y_log:
            y_last_p = np.exp(y_last_p)-1
        last_day_scores = np.append( last_day_scores, [rmse(y_last_day, y_last_p)] )
            
        train_df     = train_df + train[feats_selected][0:-nr-25]
        last_days_df = last_days_df + train[feats_selected][-nr-25:-25]
        y_train_all  = np.concatenate((y_train_all, train.pm25[0:-nr-25].values))
        y_last_days  = np.concatenate((y_last_days, y_last_day))
        y_last_pred  = np.concatenate((y_last_pred, y_last_p))
        y_pred_all   = np.concatenate((y_pred_all, y_pred))
        
        # CV
        
        side = 2  # pozwala uniknąć cyklicznych podobieństw parametru - dla 2 sprawdza 5 sąsiednich wartosci
        part_scores = np.array([])
        for i in np.arange(nr):                         # waliduje każdy model osobno
            scores = np.array([])
            for shift in np.arange(-i-side, -i+side):         # sprawdza przesunięcia wykresu - kiedy bład będzie najmniejszy?
                X_cv = train[feats_selected][0:-25].values
                y_cv = train.pm25[0:-25].values
                if y_log:
                    y_cv = train.pm25_log[0:-25].values
                y_cv = pd.Series(y_cv).shift(shift).values    # przesuwa y w przeszłośc (wartości ujemne)
                y_pred_cv = (models[i].predict(X_cv)).astype(np.float64)
                if y_log: 
                    y_cv = np.exp(y_cv)-1
                    y_pred_cv = np.exp(y_pred_cv)-1
                scores = np.append( scores, [rmse(y_cv[side:-side-nr], y_pred_cv[side:-side-nr])] )
            part_scores = np.append( part_scores, [scores[scores.argmin()]] )
#         print(part_scores.mean(), part_scores.std())

        global_scores = np.append( global_scores, part_scores )
    r2_total = r2_score(y_last_days, y_last_pred)
    rmse_total = rmse(y_last_days, y_last_pred)
    print('shift_score: {:.3f}, last_24_score: {:.3f}, rmse: {:.3f}, r2: {:.3f}\n'.format(global_scores.mean(),
                                                                                        last_day_scores.mean(),
                                                                                        rmse_total, r2_total))      
    return (global_scores, last_day_scores, rmse_total, r2_total, models, y_pred_all, train_df, last_days_df, 
            y_train_all, y_last_days, y_last_pred, feats_selected)

In [6]:
%%time

# ostani dzień w zbiorze treningowym - model finałowy

name = 'avg_18.p'
log_y = False
df, train, test, parts, num_feats, omit_feats, time_feats, black_list_feats = \
    feature_engineering_aggr(avg_len=6, roll_nr=[24], agr_fn=['median'])
feats = num_feats
print(df.shape, train.shape, test.shape, len(parts), len(feats))
with_params = {
    'model_selected': xgb.XGBRegressor,
    'model_params'  : {'learning_rate': 0.2476843816313341, 
                         'max_depth': 5, 
                         'n_estimators': 100, 
                         'n_jobs': 8, 
                         'objective': 'reg:squarederror', 
                         'random_state': 70, 
                         'subsample': 0.8561623057645249, 
                        },
    'parts_to_go'     : parts, 
    'feats_selected'  : feats,
    'y_log'           : log_y,
    'nr'              : 24
}
global_scores, last_day_scores, rmse_total, r2_total, models, y_pred_all, train_df, last_days_df, \
            y_train_all, y_last_days, y_last_pred, feats_selected = run_model_nr(**with_params)

result = global_scores, last_day_scores, rmse_total, r2_total, models, y_pred_all, train_df, last_days_df, \
            y_train_all, y_last_days, y_last_pred, feats_selected
with open(name, 'wb') as f:
    pickle.dump(result, f)
test['pm25'] = y_pred_all
test[ ['id', 'pm25'] ].to_csv('late_nr_{}.csv'.format(name[:-2]), index=False)

(40152, 61) (38952, 61) (1200, 61) 50 50



shift_score: 0.382, last_24_score: 3.490, rmse: 4.007, r2: 0.954

Wall time: 5min 32s


In [7]:
# poszukiwanie optymalnych parametrów dla modeli agregacyjnych, walidacja last day
# bez uwzglednienia przesunięcia cech

def space_to_param(space):
    int_param = ['max_depth', 'random_state', 'min_child_weight', 'n_estimators', 'n_jobs']
    
    params  = {}
    for x in space:
        if x == 'y_log':            y_log  = space[x]
        if x == 'avg_len':        avg_len  = int(space[x])
        if x == 'roll_nr':        roll_nr  = space[x]
        if x == 'agr_fn':          agr_fn  = space[x]    
        if x in int_param:       params[x] = int(space[x])   
        else:                    params[x] = space[x]            
    return y_log, params, roll_nr, agr_fn, avg_len

def param_opt(space):

    y_log, params, roll_nr, agr_fn, avg_len = space_to_param(space)
    
    df, train, test, parts, num_feats, omit_feats, time_feats, black_list_feats = \
        feature_engineering_aggr(avg_len=avg_len, roll_nr=roll_nr, agr_fn=agr_fn)
    feats = num_feats
    
    with_params = {
        'model_params'   : params,
        'parts_to_go'    : parts,
        'y_log'          : y_log,
        'feats_selected' : feats,
        'nr'             : 24,
        'model_selected' : xgb.XGBRegressor,
    }
      
    global_scores, last_day_scores, rmse_total, r2_total, models, y_pred_all, train_df, last_days_df, \
            y_train_all, y_last_days, y_last_pred, feats_selected =  run_model_day(**with_params)
    print(params, '_\n')

    return{'loss': last_day_scores.mean(), 'status': STATUS_OK }

space ={
    'subsample'      : hp.uniform('x_subsample', 0.7, 1.),
    'learning_rate'  : hp.uniform('x_learning_rate', 0.05, 0.3),
    'random_state'   : hp.choice('x_random_state', range(1, 1000, 1)),
    'max_depth'      : hp.choice('x_max_depth', range(5, 8, 1)),
    'roll_nr'        : hp.choice('x_roll_nr', [[6], [12], [18], [24], [6, 12], [12, 18], [18, 24], \
                                               [6, 24], [6, 18], [12, 24]]),
    'agr_fn'         : hp.choice('x_agr_fn', [['mean'], ['median'], ['mean', 'median']]),
    'n_estimators'   : hp.choice('x_n_estimators', range(50, 300, 50)),
    'objective'      : 'reg:squarederror',
    'n_jobs'         : -1,
    'y_log'          : hp.choice('x_y_log', [True, False]),
    'avg_len'        : hp.choice('x_avg_len', [6, 12, 18, 24]),
}

trials = Trials()
best_params = fmin(fn=param_opt,
            space=space,
            algo=partial(tpe.suggest, n_startup_jobs=1),
            max_evals=1,
            trials=trials)

print("The best params: ", best_params)

  0%|                                                                            | 0/1 [00:00<?, ?trial/s, best loss=?]

shift_score: 0.076, last_24_score: 4.211, rmse: 5.691, r2: 0.907                                                       

{'agr_fn': ('median',), 'avg_len': 24, 'learning_rate': 0.2684062129677209, 'max_depth': 7, 'n_estimators': 100, 'n_jobs': -1, 'objective': 'reg:squarederror', 'random_state': 582, 'roll_nr': (24,), 'subsample': 0.8614156053325035, 'y_log': True}
_                                                                                                                      

100%|██████████████████████████████████████████████████| 1/1 [08:20<00:00, 500.26s/trial, best loss: 4.210761487900741]
The best params:  {'x_agr_fn': 1, 'x_avg_len': 3, 'x_learning_rate': 0.2684062129677209, 'x_max_depth': 2, 'x_n_estimators': 1, 'x_random_state': 581, 'x_roll_nr': 3, 'x_subsample': 0.8614156053325035, 'x_y_log': 0}


In [6]:
# przesuwa dane które znamy w przyszłości (zmienne zależne od czasu, prognozy, ewentualnie wyliczone średnie np. dla dni tygodnia)
# muszą być określone w zbiorze testowym

def shift_X(X, feats, nr=-1):
    for feat in feats:
        X[feat] = X[feat].shift(nr)
    return X

In [7]:
# ostatni dzień w zbiorze treningowym
# przesunięcie danych znanych w przyszłości

def run_model_nr_X_shift(nr, model_selected, model_params, parts_to_go, feats_selected, time_feats, y_log):
    
    train_df, y_train_all = pd.DataFrame(columns=feats_selected), np.array([])
    last_days_df, y_last_days, y_last_pred, y_pred_all = train_df.copy(), np.array([]), np.array([]), np.array([])
    models, global_scores, last_day_scores = [], np.array([]), np.array([])

    for i in np.arange(nr):
        models.append(model_selected(**model_params))          # odzielne modele dla 24h
    
    for train in tqdm(parts_to_go):                               # kolejne bloki danych
  
        X = train[feats_selected]
        y = (train['pm25'].values).astype(np.float64)
        if y_log:
            y = (train['pm25_log'].values).astype(np.float64)
        
        for i in np.arange(nr):                             # każdy model szkolony oddzielnie
            X = shift_X(X, time_feats, -1)                  # przesuwa kolumny znane w przyszłości - time_feats
            X_last  = X[-25:-24].values                            # ostatni wiersz z bloku danych - do predykcji
            X_train = X[0:-25].values                              # pomija zbiór testowy i ostatni wiersz
            y = pd.Series(y).shift(-1).values               # przesuwa pm25 o 1h w przeszłość
            y_train = y[0:-25]                       
            models[i].fit(X_train, y_train)                        
            y_last = (models[i].predict(X_last)).astype(np.float64)
            y[-25] = y_last[0]                                  # prognoza na 1h
        
        y_pred = y[-nr-24:-24]                                  # prognoza na 24h
        if y_log:
            y_pred = np.exp(y_pred)-1
         
        #last day CV
        
        X = train[feats_selected]
        y_last_day = train.pm25[-nr-25:-25].values
        y_last_p = np.array([])
        for i in np.arange(nr):
            X = shift_X(X, time_feats, -1)
            X_last_day = X[-nr-25:-nr-24].values                           # godzina poprzedzająca ostatni dzień z danymi
            y_hour = (models[i].predict(X_last_day)).astype(np.float64)
            y_last_p = np.concatenate((y_last_p, y_hour))
        if y_log:
            y_last_p = np.exp(y_last_p)-1
        last_day_scores = np.append( last_day_scores, [rmse(y_last_day, y_last_p)] )
            
        train_df     = train_df + train[feats_selected][0:-25]
        last_days_df = last_days_df + train[feats_selected][-nr-25:-25]
        y_train_all  = np.concatenate((y_train_all, train.pm25[0:-25].values))
        y_last_days  = np.concatenate((y_last_days, y_last_day))
        y_last_pred  = np.concatenate((y_last_pred, y_last_p))
        y_pred_all   = np.concatenate((y_pred_all, y_pred))
        
        # CV
        
        side = 2  # pozwala uniknąć cyklicznych podobieństw parametru - dla 2 sprawdza 5 sąsiednich wartosci
        part_scores = np.array([])
        for i in np.arange(nr):                         # waliduje każdy model osobno
            scores = np.array([])
            for shift in np.arange(-i-side, -i+side):   # przesunięcia wykresu - kiedy bład będzie najmniejszy?
                X_cv = train[feats_selected][0:-25]
                y_cv = train.pm25[0:-25].values
                if y_log:
                    y_cv = train.pm25_log[0:-25].values
                y_cv = pd.Series(y_cv).shift(shift).values    # przesuwa y w przeszłośc (wartości ujemne)
                X_cv = shift_X(X_cv, time_feats, shift)
                y_pred_cv = (models[i].predict(X_cv.values)).astype(np.float64)
                if y_log: 
                    y_cv = np.exp(y_cv)-1
                    y_pred_cv = np.exp(y_pred_cv)-1
                scores = np.append( scores, [rmse(y_cv[side:-side-nr], y_pred_cv[side:-side-nr])] )
            part_scores = np.append( part_scores, [scores[scores.argmin()]] )
#         print(part_scores.mean(), part_scores.std())

        global_scores = np.append( global_scores, part_scores )
    r2_total = r2_score(y_last_days, y_last_pred)
    rmse_total = rmse(y_last_days, y_last_pred)
    print('shift_score: {:.3f}, last_24_score: {:.3f}, rmse: {:.3f}, r2: {:.3f}\n'.format(global_scores.mean(),
                                                                                        last_day_scores.mean(),
                                                                                        rmse_total, r2_total))      
    return (global_scores, last_day_scores, rmse_total, r2_total, models, y_pred_all, train_df, last_days_df, 
            y_train_all, y_last_days, y_last_pred, feats_selected)

### uwzgledniając przesunięcie kolumn zależnych od czasu poprawa wyniku do 12.96040 z 13.33315

In [10]:
%%time

name = 'X_shift_avg_18.p'
log_y = False
df, train, test, parts, num_feats, omit_feats, time_feats, black_list_feats = \
    feature_engineering_aggr(avg_len=6, roll_nr=[24], agr_fn=['median'])
feats = num_feats
print(df.shape, train.shape, test.shape, len(parts), len(feats))
with_params = {
    'model_selected': xgb.XGBRegressor,
    'model_params'  : {'learning_rate': 0.2476843816313341, 
                         'max_depth': 5, 
                         'n_estimators': 100, 
                         'n_jobs': -1, 
                         'objective': 'reg:squarederror', 
                         'random_state': 70, 
                         'subsample': 0.8561623057645249, 
                        },
    'parts_to_go'     : parts, 
    'feats_selected'  : feats,
    'time_feats'      : time_feats,
    'y_log'           : log_y,
    'nr'              : 24
}
global_scores, last_day_scores, rmse_total, r2_total, models, y_pred_all, train_df, last_days_df, \
            y_train_all, y_last_days, y_last_pred, feats_selected = run_model_nr_X_shift(**with_params)

result = global_scores, last_day_scores, rmse_total, r2_total, models, y_pred_all, train_df, last_days_df, \
            y_train_all, y_last_days, y_last_pred, feats_selected
with open(name, 'wb') as f:
    pickle.dump(result, f)
test['pm25'] = y_pred_all
test[ ['id', 'pm25'] ].to_csv('late_nr_{}.csv'.format(name[:-2]), index=False)

(40152, 61) (38952, 61) (1200, 61) 50 50



shift_score: 0.381, last_24_score: 3.500, rmse: 4.009, r2: 0.954

Wall time: 5min 57s


In [8]:
# # ostatni dzień pominięty w zbiorze treningowym - użyty do walidacji, stosujemy do hyperopt

def run_model_day_X_shift(nr, model_selected, model_params, parts_to_go, feats_selected, time_feats, y_log):
    
    train_df, y_train_all = pd.DataFrame(columns=feats_selected), np.array([])
    last_days_df, y_last_days, y_last_pred, y_pred_all = train_df.copy(), np.array([]), np.array([]), np.array([])
    models, global_scores, last_day_scores = [], np.array([]), np.array([])

    for i in np.arange(nr):
        models.append(model_selected(**model_params))          # odzielne modele dla 24h
    
    for train in tqdm(parts_to_go):                               # kolejne bloki danych
  
        X = train[feats_selected]
        y = (train['pm25'].values).astype(np.float64)
        if y_log:
            y = (train['pm25_log'].values).astype(np.float64)
        
        for i in np.arange(nr):                             # każdy model szkolony oddzielnie
            X = shift_X(X, time_feats, -1)                  # przesuwa kolumny znane w przyszłości - time_feats
            X_last  = X[-25:-24].values                     # ostatni wiersz z bloku danych - do predykcji
            X_train = X[0:-nr-25].values                    # pomija zbiór testowy, ostatni dzień i ostatni wiersz
            y = pd.Series(y).shift(-1).values               # przesuwa pm25 o 1h w przeszłość
            y_train = y[0:-nr-25]                     
            models[i].fit(X_train, y_train)                        
            y_last = (models[i].predict(X_last)).astype(np.float64)
            y[-25] = y_last[0]                                  # prognoza na 1h
        
        y_pred = y[-nr-24:-24]                                  # prognoza na 24h
        if y_log:
            y_pred = np.exp(y_pred)-1
         
        #last day CV
        
        X = train[feats_selected]
        y_last_day = train.pm25[-nr-25:-25].values
        y_last_p = np.array([])
        for i in np.arange(nr):
            X = shift_X(X, time_feats, -1)
            X_last_day = X[-nr-25:-nr-24].values                           # godzina poprzedzająca ostatni dzień z danymi
            y_hour = (models[i].predict(X_last_day)).astype(np.float64)
            y_last_p = np.concatenate((y_last_p, y_hour))
        if y_log:
            y_last_p = np.exp(y_last_p)-1
        last_day_scores = np.append( last_day_scores, [rmse(y_last_day, y_last_p)] )
            
        train_df     = train_df + train[feats_selected][0:-nr-25]
        last_days_df = last_days_df + train[feats_selected][-nr-25:-25]
        y_train_all  = np.concatenate((y_train_all, train.pm25[0:-nr-25].values))
        y_last_days  = np.concatenate((y_last_days, y_last_day))
        y_last_pred  = np.concatenate((y_last_pred, y_last_p))
        y_pred_all   = np.concatenate((y_pred_all, y_pred))
        
        # CV
        
        side = 2  # pozwala uniknąć cyklicznych podobieństw parametru - dla 2 sprawdza 5 sąsiednich wartosci
        part_scores = np.array([])
        for i in np.arange(nr):                         # waliduje każdy model osobno
            scores = np.array([])
            for shift in np.arange(-i-side, -i+side):   # przesunięcia wykresu - kiedy bład będzie najmniejszy?
                X_cv = train[feats_selected][0:-nr-25]
                y_cv = train.pm25[0:-nr-25].values
                if y_log:
                    y_cv = train.pm25_log[0:-nr-25].values
                y_cv = pd.Series(y_cv).shift(shift).values    # przesuwa y w przeszłośc (wartości ujemne)
                X_cv = shift_X(X_cv, time_feats, shift)
                y_pred_cv = (models[i].predict(X_cv.values)).astype(np.float64)
                if y_log: 
                    y_cv = np.exp(y_cv)-1
                    y_pred_cv = np.exp(y_pred_cv)-1
                scores = np.append( scores, [rmse(y_cv[side:-side-nr], y_pred_cv[side:-side-nr])] )
            part_scores = np.append( part_scores, [scores[scores.argmin()]] )
#         print(part_scores.mean(), part_scores.std())

        global_scores = np.append( global_scores, part_scores )
    r2_total = r2_score(y_last_days, y_last_pred)
    rmse_total = rmse(y_last_days, y_last_pred)
    print('shift_score: {:.3f}, last_24_score: {:.3f}, rmse: {:.3f}, r2: {:.3f}\n'.format(global_scores.mean(),
                                                                                        last_day_scores.mean(),
                                                                                        rmse_total, r2_total))      
    return (global_scores, last_day_scores, rmse_total, r2_total, models, y_pred_all, train_df, last_days_df, 
            y_train_all, y_last_days, y_last_pred, feats_selected)

In [10]:
# poszukiwanie optymalnych parametrów dla modeli agregacyjnych, walidacja last day
# uwzglednienia przesunięcie cech znanych w przyszłości

def space_to_param(space):
    int_param = ['max_depth', 'random_state', 'min_child_weight', 'n_estimators', 'n_jobs']
    
    params  = {}
    for x in space:
        if x == 'y_log':            y_log  = space[x]
        if x == 'avg_len':        avg_len  = int(space[x])
        if x == 'roll_nr':        roll_nr  = space[x]
        if x == 'agr_fn':          agr_fn  = space[x]    
        if x in int_param:       params[x] = int(space[x])   
        else:                    params[x] = space[x]            
    return y_log, params, roll_nr, agr_fn, avg_len

def param_opt(space):

    y_log, params, roll_nr, agr_fn, avg_len = space_to_param(space)
    
    df, train, test, parts, num_feats, omit_feats, time_feats, black_list_feats = \
        feature_engineering_aggr(avg_len=avg_len, roll_nr=roll_nr, agr_fn=agr_fn)
    feats = num_feats
    print("{} {} {} {} {}".format(df.shape, train.shape, test.shape, len(parts), len(feats)))
    with_params = {
        'model_params'   : params,
        'parts_to_go'    : parts,
        'y_log'          : y_log,
        'feats_selected' : feats,
        'time_feats'     : time_feats,
        'nr'             : 24,
        'model_selected' : xgb.XGBRegressor,
    }
      
    global_scores, last_day_scores, rmse_total, r2_total, models, y_pred_all, train_df, last_days_df, \
            y_train_all, y_last_days, y_last_pred, feats_selected = run_model_day_X_shift(**with_params)
    print("{}".format(params))
    
    return{'loss': rmse_total, 'status': STATUS_OK }

space ={
    'subsample'      : hp.uniform('x_subsample', 0.7, 1.),
    'learning_rate'  : hp.uniform('x_learning_rate', 0.05, 0.3),
    'random_state'   : hp.choice('x_random_state', [123]),
    'max_depth'      : hp.choice('x_max_depth', [4, 5]),
    'roll_nr'        : hp.choice('x_roll_nr', [[6], [12], [18], [24], [6, 12], [12, 18], [18, 24], \
                                               [6, 24], [6, 18], [12, 24]]),
    'agr_fn'         : hp.choice('x_agr_fn', [['median']]),
    'n_estimators'   : hp.choice('x_n_estimators', [50, 100]),
    'objective'      : 'reg:squarederror',
    'n_jobs'         : 8,
    'y_log'          : hp.choice('x_y_log', [True, False]),
    'avg_len'        : hp.choice('x_avg_len', [6, 12, 18, 24]),
}

trials = Trials()
best_params = fmin(fn=param_opt,
            space=space,
            algo=partial(tpe.suggest, n_startup_jobs=1),
            max_evals=1,
            trials=trials)

print("The best params: ", best_params)

(40152, 79) (38952, 79) (1200, 79) 50 68                                                                               
  0%|                                                                            | 0/1 [00:04<?, ?trial/s, best loss=?]

shift_score: 0.785, last_24_score: 3.815, rmse: 4.876, r2: 0.932                                                       

{'agr_fn': ('median',), 'avg_len': 6, 'learning_rate': 0.1279118891350392, 'max_depth': 5, 'n_estimators': 100, 'n_jobs': 8, 'objective': 'reg:squarederror', 'random_state': 123, 'roll_nr': (6, 12), 'subsample': 0.9729745085631428, 'y_log': False}
100%|██████████████████████████████████████████████████| 1/1 [06:32<00:00, 392.16s/trial, best loss: 4.876393392308629]
The best params:  {'x_agr_fn': 0, 'x_avg_len': 0, 'x_learning_rate': 0.1279118891350392, 'x_max_depth': 1, 'x_n_estimators': 1, 'x_random_state': 0, 'x_roll_nr': 4, 'x_subsample': 0.9729745085631428, 'x_y_log': 1}
